##### Copyright 2019 The TensorFlow Neural Structured Learning Authors

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用计算图正则化实现利用自然计算图的文档分类

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/neural_structured_learning/tutorials/graph_keras_mlp_cora"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/neural-structured-learning/blob/master/g3doc/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>      <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/neural_structured_learning/tutorials/graph_keras_mlp_cora.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">下载笔记本</a>
</td>
</table>

## 概述

计算图正则化是一种在更广义的神经计算图学习 ([Bui et al., 2018](https://research.google/pubs/pub46568.pdf)) 范式下应用的特定技术。核心思想是利用计算图正则化目标，在兼具带标签数据和无标签数据的情况下训练神经网络模型。

在本教程中，我们将了解如何使用计算图正则化对构成自然（有机）计算图的文档进行分类。

使用神经结构学习 (NSL) 框架创建计算图正则化模型的一般方法如下：

1. 基于输入计算图和样本特征生成训练数据。计算图中的节点对应于样本，计算图中的边对应于样本对之间的相似度。除原始节点特征外，所得的训练数据还将包含近邻特征。
2. 使用 `Keras` 序列式、函数式或子类 API 作为基础模型创建神经网络。
3. 使用 NSL 框架提供的 **`GraphRegularization`** 包装器类包装基础模型，以创建新的计算图 `Keras` 模型。这个新模型将包含计算图正则化损失作为其训练目标中的一个正规化项。
4. 训练和评估计算图 `Keras` 模型。

## 设置


安装 Neural Structured Learning 软件包。

In [ ]:
!pip install --quiet neural-structured-learning

## 依赖项和导入

In [ ]:
import neural_structured_learning as nsl

import tensorflow as tf

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Cora 数据集

[Cora 数据集](https://linqs.soe.ucsc.edu/data)是一种引用计算图，其节点代表机器学习论文，边代表论文对之间的引用。涉及的任务为文档分类，目标是将每篇论文归纳到 7 个类别中的一类。换言之，这是一种包含 7 个类的多类分类问题。

### 计算图

原始计算图是有向计算图。但在本示例中，我们考虑采用此计算图的无向版本。因此，如果论文 A 引用了论文 B，我们认为论文 B 也引用了论文 A。尽管这不一定正确，但在本示例中，我们将引用视为相似度的代名词，而相似度通常具备可交换性质。

### 特征

输入中的每篇论文实际上都包含 2 个特征：

1. **单词**：论文中文本的密集型多热词袋表示。Cora 数据集的词汇中包含 1433 个唯一单词。因此，此特征的长度为 1433，位置“i”的值为 0/1，指示给定的论文中是否存在词汇中的单词“i”。

2. **标签**：用于表示论文的类 ID（类别）的单个整数。

### 下载 Cora 数据集

In [ ]:
!wget --quiet -P /tmp https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
!tar -C /tmp -xvzf /tmp/cora.tgz

### 将 Cora 数据转换为 NSL 格式

为了预处理 Cora 数据集并将其转换为神经结构学习所需的格式，我们将运行 **'preprocess_cora_dataset.py'** 脚本，NSL GitHub 仓库中包含此脚本。此脚本可执行以下操作：

1. 使用原始节点特征和计算图生成近邻特征。
2. 生成包含 `tf.train.Example` 实例的训练和测试数据拆分。
3. 将生成的训练和测试数据保存为 `TFRecord` 格式。

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/neural-structured-learning/master/neural_structured_learning/examples/preprocess/cora/preprocess_cora_dataset.py

!python preprocess_cora_dataset.py \
--input_cora_content=/tmp/cora/cora.content \
--input_cora_graph=/tmp/cora/cora.cites \
--max_nbrs=5 \
--output_train_data=/tmp/cora/train_merged_examples.tfr \
--output_test_data=/tmp/cora/test_examples.tfr

## 全局变量

训练和测试数据的文件路径基于用于调用上述 **'preprocess_cora_dataset.py'** 脚本的命令行标记值。

In [ ]:
### Experiment dataset
TRAIN_DATA_PATH = '/tmp/cora/train_merged_examples.tfr'
TEST_DATA_PATH = '/tmp/cora/test_examples.tfr'

### Constants used to identify neighbor features in the input.
NBR_FEATURE_PREFIX = 'NL_nbr_'
NBR_WEIGHT_SUFFIX = '_weight'

## 超参数

我们将使用 `HParams` 的实例来包含用于训练和评估的各种超参数和常量。以下为各项内容的简要介绍：

- **num_classes**：共有 7 个不同的类

- **max_seq_length**：此参数为词汇的大小，输入中的所有实例均具有密集型多热词袋表示。换言之，单词的值为 1 表示输入中存在该单词，值为 0 则表示不存在该单词。

- **distance_type**：此参数为用于正则化样本与其近邻的距离指标。

- **graph_regularization_multiplier**：此参数控制计算图正则化项在总体损失函数中的相对权重。

- **num_neighbors**：用于计算图正则化的近邻数。该值必须小于或等于运行 `preprocess_cora_dataset.py` 时上文使用的 `max_nbrs` 命令行参数。

- **num_fc_units**：神经网络中的全连接层的数量。

- **train_epochs**：训练周期数。

- **batch_size**：用于训练和评估的批次大小。

- **dropout_rate**：控制每个全连接层之后的随机失活率

- **eval_steps**：认定评估完成之前需要处理的批次数。如果设置为 `None`，则将评估测试集中的所有实例。

In [ ]:
class HParams(object):
  """Hyperparameters used for training."""
  def __init__(self):
    ### dataset parameters
    self.num_classes = 7
    self.max_seq_length = 1433
    ### neural graph learning parameters
    self.distance_type = nsl.configs.DistanceType.L2
    self.graph_regularization_multiplier = 0.1
    self.num_neighbors = 1
    ### model architecture
    self.num_fc_units = [50, 50]
    ### training parameters
    self.train_epochs = 100
    self.batch_size = 128
    self.dropout_rate = 0.5
    ### eval parameters
    self.eval_steps = None  # All instances in the test set are evaluated.

HPARAMS = HParams()

## 加载训练和测试数据

如此笔记本的前文所述，输入训练和测试数据已由 **'preprocess_cora_dataset.py'** 创建。我们将把它们加载到两个 `tf.data.Dataset` 对象中：一个用于训练，另一个用于测试。

在模型的输入层中，我们不仅要从每个样本中提取“单词”和“标签”特征，还将基于 `hparams.num_neighbors` 值提取相应的近邻特征。对于近邻数少于 `hparams.num_neighbors` 的实例，将为那些不存在的近邻特征分配虚拟值。

In [ ]:
def make_dataset(file_path, training=False):
  """Creates a `tf.data.TFRecordDataset`.

  Args:
    file_path: Name of the file in the `.tfrecord` format containing
      `tf.train.Example` objects.
    training: Boolean indicating if we are in training mode.

  Returns:
    An instance of `tf.data.TFRecordDataset` containing the `tf.train.Example`
    objects.
  """

  def parse_example(example_proto):
    """Extracts relevant fields from the `example_proto`.

    Args:
      example_proto: An instance of `tf.train.Example`.

    Returns:
      A pair whose first value is a dictionary containing relevant features
      and whose second value contains the ground truth label.
    """
    # The 'words' feature is a multi-hot, bag-of-words representation of the
    # original raw text. A default value is required for examples that don't
    # have the feature.
    feature_spec = {
        'words':
            tf.io.FixedLenFeature([HPARAMS.max_seq_length],
                                  tf.int64,
                                  default_value=tf.constant(
                                      0,
                                      dtype=tf.int64,
                                      shape=[HPARAMS.max_seq_length])),
        'label':
            tf.io.FixedLenFeature((), tf.int64, default_value=-1),
    }
    # We also extract corresponding neighbor features in a similar manner to
    # the features above during training.
    if training:
      for i in range(HPARAMS.num_neighbors):
        nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, i, 'words')
        nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, i,
                                         NBR_WEIGHT_SUFFIX)
        feature_spec[nbr_feature_key] = tf.io.FixedLenFeature(
            [HPARAMS.max_seq_length],
            tf.int64,
            default_value=tf.constant(
                0, dtype=tf.int64, shape=[HPARAMS.max_seq_length]))

        # We assign a default value of 0.0 for the neighbor weight so that
        # graph regularization is done on samples based on their exact number
        # of neighbors. In other words, non-existent neighbors are discounted.
        feature_spec[nbr_weight_key] = tf.io.FixedLenFeature(
            [1], tf.float32, default_value=tf.constant([0.0]))

    features = tf.io.parse_single_example(example_proto, feature_spec)

    label = features.pop('label')
    return features, label

  dataset = tf.data.TFRecordDataset([file_path])
  if training:
    dataset = dataset.shuffle(10000)
  dataset = dataset.map(parse_example)
  dataset = dataset.batch(HPARAMS.batch_size)
  return dataset


train_dataset = make_dataset(TRAIN_DATA_PATH, training=True)
test_dataset = make_dataset(TEST_DATA_PATH)

让我们看看训练数据集中的内容。

In [ ]:
for feature_batch, label_batch in train_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  nbr_feature_key = '{}{}_{}'.format(NBR_FEATURE_PREFIX, 0, 'words')
  nbr_weight_key = '{}{}{}'.format(NBR_FEATURE_PREFIX, 0, NBR_WEIGHT_SUFFIX)
  print('Batch of neighbor inputs:', feature_batch[nbr_feature_key])
  print('Batch of neighbor weights:',
        tf.reshape(feature_batch[nbr_weight_key], [-1]))
  print('Batch of labels:', label_batch)

让我们看看测试数据集中的内容。

In [ ]:
for feature_batch, label_batch in test_dataset.take(1):
  print('Feature list:', list(feature_batch.keys()))
  print('Batch of inputs:', feature_batch['words'])
  print('Batch of labels:', label_batch)

## 模型定义

为了演示如何使用计算图正则化，我们首先为此问题构建基础模型。我们将使用一个简单的前馈神经网络，其中包含 2 个隐藏层，之间应用随机失活。我们将展示使用 `tf.Keras` 框架支持的所有模型类型（序贯、函数式和子类）创建基础模型的过程。

### 序贯基础模型

In [ ]:
def make_mlp_sequential_model(hparams):
  """Creates a sequential multi-layer perceptron model."""
  model = tf.keras.Sequential()
  model.add(
      tf.keras.layers.InputLayer(
          input_shape=(hparams.max_seq_length,), name='words'))
  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  model.add(
      tf.keras.layers.Lambda(lambda x: tf.keras.backend.cast(x, tf.float32)))
  for num_units in hparams.num_fc_units:
    model.add(tf.keras.layers.Dense(num_units, activation='relu'))
    # For sequential models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    model.add(tf.keras.layers.Dropout(hparams.dropout_rate))
  model.add(tf.keras.layers.Dense(hparams.num_classes))
  return model

### 函数式基础模型

In [ ]:
def make_mlp_functional_model(hparams):
  """Creates a functional API-based multi-layer perceptron model."""
  inputs = tf.keras.Input(
      shape=(hparams.max_seq_length,), dtype='int64', name='words')

  # Input is already one-hot encoded in the integer format. We cast it to
  # floating point format here.
  cur_layer = tf.keras.layers.Lambda(
      lambda x: tf.keras.backend.cast(x, tf.float32))(
          inputs)

  for num_units in hparams.num_fc_units:
    cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
    # For functional models, by default, Keras ensures that the 'dropout' layer
    # is invoked only during training.
    cur_layer = tf.keras.layers.Dropout(hparams.dropout_rate)(cur_layer)

  outputs = tf.keras.layers.Dense(hparams.num_classes)(cur_layer)

  model = tf.keras.Model(inputs, outputs=outputs)
  return model

### 子类基础模型

In [ ]:
def make_mlp_subclass_model(hparams):
  """Creates a multi-layer perceptron subclass model in Keras."""

  class MLP(tf.keras.Model):
    """Subclass model defining a multi-layer perceptron."""

    def __init__(self):
      super(MLP, self).__init__()
      # Input is already one-hot encoded in the integer format. We create a
      # layer to cast it to floating point format here.
      self.cast_to_float_layer = tf.keras.layers.Lambda(
          lambda x: tf.keras.backend.cast(x, tf.float32))
      self.dense_layers = [
          tf.keras.layers.Dense(num_units, activation='relu')
          for num_units in hparams.num_fc_units
      ]
      self.dropout_layer = tf.keras.layers.Dropout(hparams.dropout_rate)
      self.output_layer = tf.keras.layers.Dense(hparams.num_classes)

    def call(self, inputs, training=False):
      cur_layer = self.cast_to_float_layer(inputs['words'])
      for dense_layer in self.dense_layers:
        cur_layer = dense_layer(cur_layer)
        cur_layer = self.dropout_layer(cur_layer, training=training)

      outputs = self.output_layer(cur_layer)

      return outputs

  return MLP()

## 创建基础模型

In [ ]:
# Create a base MLP model using the functional API.
# Alternatively, you can also create a sequential or subclass base model using
# the make_mlp_sequential_model() or make_mlp_subclass_model() functions
# respectively, defined above. Note that if a subclass model is used, its
# summary cannot be generated until it is built.
base_model_tag, base_model = 'FUNCTIONAL', make_mlp_functional_model(HPARAMS)
base_model.summary()

## 训练基础 MLP 模型

In [ ]:
# Compile and train the base MLP model
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
base_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

## 评估基础 MLP 模型

In [ ]:
# Helper function to print evaluation metrics.
def print_metrics(model_desc, eval_metrics):
  """Prints evaluation metrics.

  Args:
    model_desc: A description of the model.
    eval_metrics: A dictionary mapping metric names to corresponding values. It
      must contain the loss and accuracy metrics.
  """
  print('\n')
  print('Eval accuracy for ', model_desc, ': ', eval_metrics['accuracy'])
  print('Eval loss for ', model_desc, ': ', eval_metrics['loss'])
  if 'graph_loss' in eval_metrics:
    print('Eval graph loss for ', model_desc, ': ', eval_metrics['graph_loss'])

In [ ]:
eval_results = dict(
    zip(base_model.metrics_names,
        base_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('Base MLP model', eval_results)

## 训练包含计算图正则化的 MLP 模型

只需短短几行代码，便可将计算图正则化整合到现有 `tf.Keras.Model` 的损失项中。包装基础模型以创建新的 `tf.Keras` 子类模型，其损失包含计算图正则化。

为了评估计算图正则化的增量收益，我们将创建一个新的基础模型实例。这是因为 `base_model` 已完成了几次训练迭代，重用这个经过训练的模型来创建计算图正则化模型对于 `base_model` 的比较而言，结果将有失偏颇。

In [ ]:
# Build a new base MLP model.
base_reg_model_tag, base_reg_model = 'FUNCTIONAL', make_mlp_functional_model(
    HPARAMS)

In [ ]:
# Wrap the base MLP model with graph regularization.
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=HPARAMS.num_neighbors,
    multiplier=HPARAMS.graph_regularization_multiplier,
    distance_type=HPARAMS.distance_type,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_reg_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
graph_reg_model.fit(train_dataset, epochs=HPARAMS.train_epochs, verbose=1)

## 评估包含计算图正则化的 MLP 模型

In [ ]:
eval_results = dict(
    zip(graph_reg_model.metrics_names,
        graph_reg_model.evaluate(test_dataset, steps=HPARAMS.eval_steps)))
print_metrics('MLP + graph regularization', eval_results)

与基础模型 (`base_model`) 相比，计算图正则化模型的准确率约高 2-3%。

## 结论

我们演示了如何使用计算图正则化在利用神经结构学习 (NSL) 框架的自然引用计算图 (Cora) 上实现文档分类。我们的[高级教程](graph_keras_lstm_imdb.ipynb)介绍了在训练包含计算图正则化的神经网络之前基于样本嵌入向量合成计算图的方法。此方法在输入不包含显式计算图时非常实用。

我们鼓励用户通过更改监督量以及对计算图正则化尝试不同的神经架构来进行进一步实验。